In [88]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [89]:
df = pd.read_csv('dataset/processed_english_twitter_data.csv', skipinitialspace=True, engine="python")
df.head()

,Lang,Text,Author,Service
0,en,card support good yet sebastian release neckla...,xYouCanCallMeAl,card
1,en,rt terraloire big bank multiple leader sit hos...,MarttinenLee,investment
2,en,rt wisdomise wisdomise main goal make financia...,Cryptullo,investment
3,en,hightoweradvsor wealth management firm based c...,AccountingToday,investment
4,en,rt warpsiwah cast emi nishino full movie http ...,Kakeksugion01,emi


In [91]:
df.groupby(by='Service').count()

,Lang,Text,Author
Service,,,
card,483,483,483
emi,92,92,92
insurance,485,485,485
investment,149,149,149
loan,421,421,421
savings,106,106,106
